In [1]:
!pip install tensorflow-model-analysis

  error: subprocess-exited-with-error
  
  pip subprocess to install build dependencies did not run successfully.
  exit code: 1
  
  [9 lines of output]
  Ignoring numpy: markers 'python_version < "3.8"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.8"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.9"' don't match your environment
    Obtaining dependency information for cython>=0.29 from https://files.pythonhosted.org/packages/7d/61/bf165c17a1296fd7db78e18fd8cbb157ab04060ec58d34ff319424af3e2d/Cython-3.0.0-cp311-cp311-win_amd64.whl.metadata
    Using cached Cython-3.0.0-cp311-cp311-win_amd64.whl.metadata (3.2 kB)
  ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
  ERROR: Could not find a version that satisfies the req


     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     - -------------------------------------- 0.1/1.9 MB 1.6 MB/s eta 0:00:02
     ----- ---------------------------------- 0.2/1.9 MB 3.0 MB/s eta 0:00:01
     ------------- -------------------------- 0.6/1.9 MB 4.8 MB/s eta 0:00:01
     ----------------------- ---------------- 1.1/1.9 MB 6.4 MB/s eta 0:00:01
     ----------------------------------- ---- 1.7/1.9 MB 7.7 MB/s eta 0:00:01
     ------------------------------------ --- 1.7/1.9 MB 6.5 MB/s eta 0:00:01
     ------------------------------------- -- 1.8/1.9 MB 5.9 MB/s eta 0:00:01
     ---------------------------------------  1.9/1.9 MB 5.2 MB/s eta 0:00:01
     ---------------------------------------- 1.9/1.9 MB 5.0 MB/s eta 0:00:00
  Obtaining dependency information for apache-beam[gcp]<3,>=2.40 from https://files.pythonhosted.org/packages/a7/8f/461ca31efd64bcc4ea0baff6fd44b1cc9079c4fe3f2fefed28fc0f064d8f/apache_beam-2.49.0-cp311-cp311-win_amd64.whl.meta

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from 

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [2]:
import matplotlib.pyplot as plt
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

In [3]:
df = pd.read_csv("train.csv")

In [4]:
df_test = pd.read_csv('test.csv')[["id", "all"]]
len(df_test)
x_df_test = df_test['all']
x_df = df['all']
x_df_temp = x_df._append(x_df_test)
y_df = df['label']
x_df_temp

0       MAJOR LIBERAL RAG RELUCTANTLY PUBLISHES Articl...
1       AT&T, Time Warner and the Death of PrivacyAT&T...
2       The World Sees a Diminished AmericaThe World S...
3       The Bolton Threat to Trump’s Middle East Polic...
4       Life at migrant centers in Germany: RT talks t...
                              ...                        
7883    Italy's ruling PD slides further in polls as e...
7884    Trump-backed Navy expansion would boost costs ...
7885    Canada and E.U. Sign Trade Deal, Bucking Resis...
7886    With Koch Brothers Academy, Conservatives Sett...
7887    More than 50 arrested for looting in Miami dur...
Name: all, Length: 17438, dtype: object

In [5]:
VOCAB_SIZE = 70000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,)
encoder.adapt(x_df)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x_df,y_df,random_state=0)


In [7]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])



ValueError: Exception encountered when calling layer "text_vectorization" (type TextVectorization).

When using `TextVectorization` to tokenize strings, the input rank must be 1 or the last shape dimension must be 1. Received: inputs.shape=(None, None) with rank=2

Call arguments received by layer "text_vectorization" (type TextVectorization):
  • inputs=tf.Tensor(shape=(None, None), dtype=string)

In [7]:
embedding_dim = 16
text_model_input = tf.keras.layers.Input(dtype=tf.string, shape=(1,))
text_model_catprocess2 = encoder(text_model_input)
text_model_embedd = tf.keras.layers.Embedding(70000, embedding_dim, name = 'embedding')(text_model_catprocess2)
text_model_bid = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(text_model_embedd)
text_model_bid2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(text_model_bid)
text_model_dense2 = tf.keras.layers.Dense(64, activation = 'relu')(text_model_bid2)
text_model_output = tf.keras.layers.Dense(1, activation = 'sigmoid')(text_model_dense2)
model = tf.keras.Model(text_model_input, text_model_output)

In [8]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [9]:
train_dataset = pd.DataFrame()
train_dataset['text'] = x_train
train_dataset['label'] = y_train

test_dataset = pd.DataFrame()
test_dataset['text'] = x_test
test_dataset['label'] = y_test

In [ ]:
history = model.fit(x_train,y_train, epochs=15,
                    validation_data=(x_test, y_test),
                    validation_steps=30)

Epoch 1/15


C:\Akshay\Tools\Python\Lib\site-packages\keras\src\backend.py:5820: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


224/224 [==============================] - 1322s 6s/step - loss: 0.4795 - accuracy: 0.8622 - val_loss: 0.3934 - val_accuracy: 0.8656
Epoch 2/15
158/224 [====================>.........] - ETA: 4:41 - loss: 0.3986 - accuracy: 0.8623

In [ ]:
x_train

In [ ]:

test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
x_df_test

In [ ]:
y_pred = model.predict(x_df_test)

In [ ]:
y_pred_rounded = y_pred.round()

In [ ]:
df_out = pd.DataFrame()
df_out['id'] = df_test['id']
df_out["label"] = y_pred_rounded.astype(int)
df_out.to_csv('submission.csv', index=False)

In [ ]:
df_out.label.value_counts()

In [ ]:
    from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
